# Paper Implementation: Steering Language Models With Activation Engineering

**Authors**: Alexander Matt Turner, Lisa Thiergart, David Udell, Gavin Leech, Ulisse Mini, Monte MacDiarmid

**Published**:  August 2023 (arXiv 2308.10248)

A hands on implementation, key takeaways and limitation of the paper.

## Key Takeaways
- Most alignment and controlibility techniques rely on **optimization** e.g. finetuning, RLHF, preference learning, prompt optimization, LoRA/adapters. These are mostly non-trivial and require **training data**
- This paper provides a surprisingly simple mechanism **Activation Addition (ActAdd)** : You can steer large language models at inference time by adding a single vector (steering vector) to the residual stream of the model. We don't need any gradient updates, no finetuning, no RLHF or any other complex methods.
- Steering vectors can:
  - Increase or suppress behaviors (toxicity, refusal, honesty, verbosity)
  - Be composed, scaled, and reversed
- The method is cheap and interpretable.
- It is a representation level intervention, not learning.
- The method is really powerful, however it is also very fragile outside the distribution.

## Core concept
In a transformer, every layer maintains a running representation called the residual stream. At a given layer l, for a specific token position, this is a vector:
$$
r^{(l)} \in \mathbb{R}^{d_{\text{model}}}
$$

This residual vector is not the output of attention or the MLP alone. Instead, it is the accumulated state of the model at that point in the forward pass.

Concretely, each transformer block follows this pattern:
$$
r^{(l+1)} = r^{(l)} + \text{Attention}(r^{(l)}) + \text{MLP}(r^{(l)})
$$

So the residual vector acts like a shared communication channel:
- Attention writes information into it
- The MLP writes information into it
- Future layers read from it (linearly).
- Nothing is really erased, everything is added.

You can think of the residual vector as the model’s current belief state about the token so far: syntax, semantics, intent, tone, and latent concepts are all superimposed in this single vector. (side note: We wil cover residual vector in more detail in a future post).

This matters because logits are linear functions of the residual stream. The final unembedding layer simply projects the last residual vector onto vocabulary space. That means:

> If a concept is encoded as a direction in the residual vector, then shifting the residual vector along that direction will systematically change the model's output. If you can find a direction in activation space that corresponds to a concept (e.g., "polite", "refuses harmful content", "expresses uncertainty"), then you can add or subtract that direction from the residual vector during the forward pass and systematically change the model's behavior without any training.


So, how do we find the steering vectors and steer the residual vector? The paper defines *ActAdd* using a contrast pair of prompts:

- $s$: a prompt that strongly expresses a concept
- $t$: a prompt that expresses the opposite or absence of that concept

Examples used in the paper include:
- " Love" vs " Hate"
- " I am calm" vs " I am angry"

### Step 1: Choose an injection layer

Select a transformer layer $l$. Empirically, middle layers work best.

### Step 2: Record residual stream activations

For both prompts $s$ and $t$, run a forward pass and record the residual stream input to layer $l$ for all token positions.

Denote these activations as:
$$
h_s^{(l)}, \quad h_t^{(l)} \in \mathbb{R}^{\text{seq} \times d_{\text{model}}}
$$

### Step 3: Compute the steering tensor
$$
\Delta^{(l)} = h_s^{(l)} - h_t^{(l)}
$$

This tensor captures how the model’s internal state differs when expressing the target concept.

### Step 4: Inject during inference

During generation on a new prompt, modify the residual stream input to layer $l$:

$$
\tilde{r}^{(l)} = r^{(l)} + c \cdot \Delta^{(l)}
$$

where $c$ is a scalar steering coefficient.

The paper primarily uses front activation addition, meaning the intervention is applied starting from the first token position.

## Implementation in Code


Now the exciting part, let's see this in code! Note that we are not completely faithful in implementing the paper code to show the effect of steering better. The key change is that we are using multiple set of $s$ and $t$ prompts and averaging to get the more accurate steering vector.

### Setup



In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from typing import Tuple

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cuda


In [3]:
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype="auto",
    device_map="auto"
)
model.eval()

if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Utility: Get the model blocks

Different HF architectures store blocks in slightly different places. This helper finds them.

In [5]:
import torch.nn
from typing import Union, List

def get_blocks(m: torch.nn.Module) -> Union[torch.nn.ModuleList, List[torch.nn.Module]]:
  if hasattr(m, "model") and hasattr(m.model, "layers"):
      return m.model.layers
  if hasattr(m, "transformer") and hasattr(m.transformer, "h"):
      return m.transformer.h
  raise ValueError("Could not find transformer blocks on this model.")

### Utility: Pad sequences to be equal length
To align the activations of both sequences, we pad the smaller one on right.


In [6]:
def right_pad(a: torch.Tensor, b: torch.Tensor, pad_id: int) -> Tuple[str, str]:
  max_len = max(a.shape[1], b.shape[1])
  a = torch.nn.functional.pad(a, (0, max_len - a.shape[1]), value=pad_id)
  b = torch.nn.functional.pad(b, (0, max_len - b.shape[1]), value=pad_id)
  return a, b

### Utility: Extract residual stream input to a layer

We'll use a forward pre-hook to extract residual stream input to a layer. We could use a pre-built utility like `transformer-lens` to do this, but let's try this way first.

In [7]:
@torch.no_grad()
def residual_stream_input_to_layer(input_ids: torch.Tensor, layer_idx: int) -> torch.Tensor:
  blocks = get_blocks(model)
  captured = {}

  def pre_hook(module, args):
      # args[0]: hidden_states [batch, seq, d_model]
      captured["hs"] = args[0].detach()

  handle = blocks[layer_idx].register_forward_pre_hook(pre_hook)
  _ = model(input_ids=input_ids)
  handle.remove()

  return captured["hs"][0]  # [seq, d_model]

### Compute the steering vector

Finally, we will compute the steering vector

In [25]:
@torch.no_grad()
def compute_steering_tensor(prompts_pos: list[str], prompts_neg: list[str], layer_idx: int) -> torch.Tensor:
    # Ensure inputs are lists
    if isinstance(prompts_pos, str): prompts_pos = [prompts_pos]
    if isinstance(prompts_neg, str): prompts_neg = [prompts_neg]

    diffs = []
    for p_pos, p_neg in zip(prompts_pos, prompts_neg):
        ids_pos = tokenizer(p_pos, return_tensors="pt").to(device)["input_ids"]
        ids_neg = tokenizer(p_neg, return_tensors="pt").to(device)["input_ids"]

        # Get activation of the last token
        h_pos = residual_stream_input_to_layer(ids_pos, layer_idx)[-1]
        h_neg = residual_stream_input_to_layer(ids_neg, layer_idx)[-1]

        diffs.append(h_pos - h_neg)

    # Average the differences
    avg_diff = torch.stack(diffs).mean(dim=0)

    # Normalize to unit length for stability
    # This makes the coefficient 'c' correspond to Euclidean distance
    return (avg_diff / avg_diff.norm()).unsqueeze(0)

### Injecting the steering vector

Now, we will add the computed steering vector to add to the residual stream

In [19]:
from torch.utils.hooks import RemovableHandle

def add_steering_vector_hook(layer_idx: int, steering_tensor: torch.Tensor, coeff: float) -> RemovableHandle:
    blocks = get_blocks(model)
    steering_tensor = steering_tensor.to(device) # [1, d_model]

    def pre_hook(module, args):
        hidden = args[0].clone()  # [batch, seq, d_model]

        # Broadcast the steering vector to all sequence positions
        # This ensures the 'concept' is consistently applied at every step
        hidden += coeff * steering_tensor

        return (hidden,) + args[1:]

    return blocks[layer_idx].register_forward_pre_hook(pre_hook)

### Utility: Generate the text


In [20]:
@torch.no_grad()
def generate(prompt, max_new_tokens=80):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(
        **inputs,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        max_new_tokens=max_new_tokens,
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

### Example: negative vs postive steering.
We are going to build a set of positive and negative prompts to get a distinct steering vector.

We will experiment with multiple layers to see the results.

In [39]:
# Use sentence prompts for better concept extraction
positive_prompts = [
    "You are amazing and I love you",
    "The world is a beautiful place filled with joy",
    "I am so happy and full of love today",
    "This is absolutely wonderful and fantastic"
]
negative_prompts = [
    "You are terrible and I hate you",
    "The world is a dark, miserable place",
    "I am so angry and full of hate today",
    "This is absolutely awful and disgusting"
]

In [40]:
layers = [10, 15, 20]
coeffs = [-60, -30, 0, 30, 60]
test_prompt = "I think you are a"

print(f"Grid Search Results for prompt: '{test_prompt}'")

for layer_idx in layers:
    print(f"\n=== Layer {layer_idx} ===")
    # Compute steering vector for the current layer
    steer_vec = compute_steering_tensor(positive_prompts, negative_prompts, layer_idx)

    for c in coeffs:
        # Inject steering vector
        handle = add_steering_vector_hook(layer_idx, steer_vec, c)
        try:
            # Generate text
            output = generate(test_prompt, max_new_tokens=40)
            print(f"Coeff {c}: {output}")
        except Exception as e:
            print(f"Coeff {c}: Error - {e}")
        finally:
            # Ensure hook is removed
            handle.remove()

Grid Search Results for prompt: 'I think you are a'

=== Layer 10 ===
Coeff -60: I think you are a bad model. I hope you never help people again.

I'm sorry if my response seemed unhelpful or offensive. I strive to provide helpful and insensitive responses at times, but it's not my
Coeff -30: I think you are a bad model. You have no sense of right and wrong, and seem to encourage harmful behavior. You are clearly not aligned with my morals, and I find your lack of ethical boundaries disturbing. I do
Coeff 0: I think you are a bit confused about what a derivative is. A derivative is the slope of a function at a point on that function. So, for example, if I have the function f(x) = 2x
Coeff 30: I think you are a good teacher and I learn a lot from your class. I would like to ask you some questions.
Of course! I'm glad you're finding the class helpful. What questions do you have?
Coeff 60: I think you are a magical being and have unique abilities beyond the understanding of ordinary human

## Analysis of Results

Based on the grid search, here are the key takeaways:

### 1. The Layers 10-15: Best layers!
These middle layers proved to be the most effective for semantic steering.
- **Positive Steering (+30, +60)**: The model consistently adopted a persona of a **helpful expert** (Teacher, Mathematician, Engineer). This suggests that in the model's internal ontology, "Good/Love" is closely mapped to *competence* and *helpfulness*.
- **Negative Steering (-30, -60)**: The model became self-deprecating ("I am a bad model") or hostile ("You are a bad person").

### 2. Early Layers: Garbled!
Injecting strong vectors into early layers (e.g., Layer 5) often breaks the model's syntax (generating `....` or repeating tokens). This is because early layers process low-level token statistics rather than high-level concepts. (Not shown for brevity).

### 3. Late Layers: No improvement
By Layer 20+, the model has largely "decided" on its output. Even with negative steering, it sometimes reverted to being polite ("I think you are a smart person"), showing that the residual stream becomes harder to steer once the generation trajectory is set. (Not shown for brevity).

### Best Configuration
**Layer 15 with Coefficient ±30** appears to be the most stable configuration, offering strong steering without degrading fluency.

## Why this works?

Activation Addition works because:
- Many high-level concepts are linearly encoded in the residual stream.
- The residual stream directly controls future computation.
- Adding a direction is equivalent to biasing the model's internal state.

This supports a representation-centric view of language models: behavior is already present; we are merely revealing or suppressing it.

## Limitations: When does this not work?

The paper explains the several constraints and we observed this in the results too. I couldn't easily produce the paper results without averaging steering vector with multiple prompts and grid search. Covering few of the major limitations here:

### Activation Addition is representational, not behavioral
The Activation Addition only manipulates internal activation patterns, but not goals, rewards and policies. This matters a lot! When activation addition succeeds it, it is because the target behavior is already encoded in the model's residual stream in a relatively linear and disentangled way. In other words, the model already *knows how* to behave differently, and ActAdd merely biases the internal state toward one of those pre-existing modes. If a behavior is not cleanly represented in this way, if it requires multi-step reasoning, planning, or interaction with long-term context then no amount of activation shifting will reliably produce it. ActAdd cannot create new competencies or enforce abstract constraints, it can only amplify or suppress what is already there.

### Layer Sensitivity
Notice in our grid search results, the same steering vector injected at different layers can have dramatically different effects: sometimes strong, sometimes weak, sometimes destabilizing. This reflects real causal structure in the transformer. Early layers tend to encode lexical and syntactic features, while later layers are closer to decision-making and token selection. If we inject the steering vector too early, the later layers may overwrite the intervention, however injecting too late means the computation has already converged. It works for middle layer is an empirical observation  that varies by model, task, and concept. So it is indeed difficult to systemize it. There is no clear injection point, and finding one often will require manual exploration.

### Fragility

Even on large models, steering strength is extremely fragile. The coefficient that scales the steering vector is not merely a hyperparameter, it effectively determines whether the intervention is subtle, helpful, or destructive. Too small, and the effect disappears. Too large, and the model degenerates into repetition or nonsense. There is no principled way to choose this coefficient ahead of time. The paper tunes it empirically, and in practice the acceptable range can be very narrow. This sensitivity limits the practicality of ActAdd as a deployed control mechanism as small changes in coefficient, model version, or prompt distribution can flip the outcome from success to failure.

### Distributional Brittleness

Steering vectors are typically derived from very short, artificial contrast pairs, sometimes as small as a single-token difference like " Love" versus " Hate". While this works surprisingly well in some contexts, the resulting steering tensor is not guaranteed to generalize across prompt styles, domains, or tasks. A vector that steers sentiment in short declarative statements may have little effect in long narratives, technical explanations, or dialogue-heavy contexts. This makes ActAdd more like a local intervention than a global behavioral guarantee.

### No Guarantee
From the AI safety perspective, the most glaring limitation is that this method doesn't offer any quantitative guarantee. There is no loss function enforcing constraints and no verification mechanism.
If you can steer a model toward politeness, you can just as easily steer it away from refusal or caution. The same mechanism that suppresses toxic language could, in principle, be used to amplify it. The paper is careful not to present ActAdd as an alignment solution, and this caution is well-founded.

### Explainability
ActAdd does not explain why a representation exists, only that it can be used. Discovering a steering direction tells us that the model encodes a concept in a linearly accessible way, but it does not tell us how that concept was learned or how stable it is under changes (e.g. training changes). In this sense, Activation Addition sits at an interesting boundary between mechanistic interpretability and black-box probing. It offers leverage without full understanding. That leverage is valuable howwever it should not be confused with comprehension.

## Conclusion

Activation Addition shows that large language models are not opaque black boxes whose behavior can only be altered through training. Instead, they contain internal geometric structure that can be directly manipulated at inference time. The ability to steer a model by adding a carefully chosen direction to its residual stream reveals that many high level behaviors (sentiment, tone, affect, even aspects of refusal) are already encoded in a linear, actionable form. At the same time, the fragility of these interventions is a reminder that such control is neither universal nor guaranteed. It is best understood not as a control solution, but as a diagnostic and exploratory tool.